In [ ]:
import pandas as pd
import os
import itertools
import datetime
import time
from tqdm import tqdm
import numpy as np

In [ ]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [ ]:
Program_Start_Time  = time.time()
print("Current file location: "+ os.getcwd())
print("Login WBI : "+ os.getlogin())

### Read Input Reference (PUI and JDA) and Do Data Preprocessing before Rolling

In [ ]:
class Data_preprocessing():
    def __init__(self):
        self.input_PUI_ref_path = r"./Input/Supply_Chain_PUI.xlsx"
        self.input_PUI_JDA_path = r"./Input/Input_JDA_List.xlsx"
    
    def Data_loading(self):
        print("File loading ....")
        #Record data loading time
        Data_loading_Start_Time = time.time()
        input_PUI_ref = pd.read_excel(self.input_PUI_ref_path, sheet_name="Raw")
        input_JDA_ref = pd.read_excel(self.input_PUI_JDA_path)
        print("Finish data loading in {} mins.".format(round((time.time() - Data_loading_Start_Time)/60, 2)))
        return input_PUI_ref, input_JDA_ref
    
    def main(self):
        df_PUI_raw, df_JDA_raw = self.Data_loading()
        print("==============================================================")
        #Record processing time
        Data_Preprocessing_Start_Time = time.time()
        #Data preprocessing and aggregation
        df_JDA_raw, MultiChip_dict = self.JDA_Input_Preprocessing(df_JDA_raw)
        df_JDA_raw_ORIGIN = df_JDA_raw[["SC Index", "ITEM_NAME", "ITEM_CATEGORY", "JDA_PRODUCED_LOC_2", "CONSUMED_ITEM", "CONSUMED_ITEM_CATEGORY", "JDA_CONSUMED_LOC_2", "RAW_PLANT"]].copy()

        df_JDA_raw = df_JDA_raw.loc[df_JDA_raw["Excluded"].isna()]
        # df_JDA_raw = df_JDA_raw.loc[:200]
        df_PUI_raw, df_agg, df_agg_final = self.Multichip_Decomposition(df_PUI_raw)
        prod = self.Prod_Consumsed_Adjacency(df_agg_final)
        Mtype = self.Mtype_Dict_Preprocessing(df_agg)
        print("Finish data preprocessing in {} mins. Begin with data rolling...\n".format(round((time.time() - Data_Preprocessing_Start_Time)/60, 2)))
        return prod, MultiChip_dict, df_JDA_raw_ORIGIN, df_JDA_raw, df_PUI_raw, df_agg, df_agg_final, Mtype
    
    def JDA_Input_Preprocessing(self, input_JDA_ref):
        # Inset SC Index for JDA input (starting from 1)
        input_JDA_ref["SC Index"] = np.arange(1, len(input_JDA_ref)+1)
        input_JDA_ref["FINAL START DATE"] = input_JDA_ref["FINAL START DATE"].astype(str).str.split(" ", expand=True)[0]
        input_JDA_ref["FINAL END DATE"] = input_JDA_ref["FINAL END DATE"].astype(str).str.split(" ", expand=True)[0]
        input_JDA_ref.insert(0, 'SC Index', input_JDA_ref.pop('SC Index'))

        MultiChip_dict = {}
        MultiChip_dict = dict(zip(input_JDA_ref["ITEM_BOM_RT_ID"], input_JDA_ref["Multi Chip"]))

        # print("\nData Schema of input JDA source: ")
        # print(input_JDA_ref.info())
        return input_JDA_ref, MultiChip_dict

    def Multichip_Decomposition(self, input_PUI_ref):
        # Transfer PUI table's index starting from 1
        input_PUI_ref.index = np.arange(1, len(input_PUI_ref) + 1)

        '''
        Create a column MutiComp_index to save the index of ICAM -> CEXX_1, CEXX_2 in per decomposition record.
        The record can be traced via the original input PUI frame to get the original multi-chip record
        '''

        # Create new column, COMP_12NC by copy "1_12NC_LIST" if that is not multi-chip (ignore ICFT), and transder the series type to string
        input_PUI_ref["SingleComp_index"] = input_PUI_ref.index
        input_PUI_ref["COMP_12NC"] = input_PUI_ref["1_12NC_LIST"].copy()
        input_PUI_ref.loc[(input_PUI_ref["COMP_12NC_LIST"].str.contains(",")) & (input_PUI_ref["TYPE"] == "ICAM"), "COMP_12NC"] = input_PUI_ref.loc[(input_PUI_ref["COMP_12NC_LIST"].str.contains(",")) & (input_PUI_ref["TYPE"] == "ICAM"), "COMP_12NC_LIST"]
        input_PUI_ref["COMP_12NC"] = input_PUI_ref["COMP_12NC"].astype(str)

        # Get component 12NC and extend them if they are multi-chip to an independent dataframe
        df_connects_multi_CEXX = input_PUI_ref.loc[input_PUI_ref["COMP_12NC"].str.contains(",")]
        df_connects_multi_CEXX.loc[df_connects_multi_CEXX["COMP_12NC"] .str.contains(","), "MutiComp"] = df_connects_multi_CEXX.loc[df_connects_multi_CEXX["COMP_12NC"].str.contains(","), "COMP_12NC"]

        df_connects_multi_CEXX["SingleComp_index"] = pd.NaT
        df_connects_multi_CEXX.loc[df_connects_multi_CEXX["COMP_12NC"].str.contains(","), "MutiComp_index"] = df_connects_multi_CEXX.loc[df_connects_multi_CEXX["COMP_12NC"].str.contains(",")].index
        df_connects_multi_CEXX["MutiComp_index"] = df_connects_multi_CEXX["MutiComp_index"].astype(int)

        def split_rows(df, column):
            # Create an empty DataFrame to store the new rows
            new_df = pd.DataFrame(columns=df.columns)

            for _, row in df.iterrows():
                # Check if the value in the specific column contains a comma
                if "," in str(row[column]):
                    # Split the value by comma
                    split_values = str(row[column]).split(',')
                    for value in split_values:
                        # Create a new row with the split value
                        new_row = row.copy()
                        new_row[column] = value.strip()  # Remove any leading/trailing whitespace
                        new_df = new_df.append(new_row, ignore_index=True)
                else:
                    # If no comma, just append the row as is
                    new_df = new_df.append(row, ignore_index=True)
            
            return new_df

        df_extend_multicomp = split_rows(df_connects_multi_CEXX, 'COMP_12NC')
        df_extend_multicomp["COMP_12NC"] = df_extend_multicomp["COMP_12NC"].astype('str')



        '''
        df_PUI_raw = original PUI table
        df_extend_multicomp = independent dataframe to save the extended CEXX connects to ICAM where ICAM is multichip (per ICAM -> CEXX per row)
        df_agg = Aggregated Result of df_PUI_raw and df_agg and reset the index.
        df_agg_final = drop (ICAM -> CEXX_1, CEXX_2) in df_agg to make the data available for rolling. SingleComp_index, MutiComp_index can be the tracing index from df_agg

        '''

        df_agg = pd.concat([input_PUI_ref, df_extend_multicomp])
        df_agg.reset_index(drop = True, inplace =True)
        df_agg.index = np.arange(1, len(df_agg) + 1)

        print("\nLength of the Original PUI table: {}".format(len(input_PUI_ref)))
        print("Length of the Extended Multi-Component PUI table: {}".format(len(df_extend_multicomp)))
        print("Length of the Aggregated PUI table: {}, {}".format(len(df_agg), len(df_agg) == (len(input_PUI_ref) + len(df_extend_multicomp))))

        df_agg_final = df_agg[~df_agg.index.isin(df_agg["MutiComp_index"].dropna())]

        print("Length of the Aggreaged PUI and drop multiple list: {}\n".format(len(df_agg_final)))

        # Dict for adjacent 12NC
        df_agg_final["PART_12NC"] = df_agg_final["PART_12NC"].astype("int64")
        df_agg_final["COMP_12NC"] = df_agg_final["COMP_12NC"].astype("int64")

        # print("\nData Schema Aggregated final PUI result: ")
        # print(df_agg_final.info())
        return input_PUI_ref, df_agg, df_agg_final
    

    def Prod_Consumsed_Adjacency(self, input_df_agg_final):
        # Dict for adjacent 12NC
        prod = {}
        for x in input_df_agg_final["PART_12NC"].unique():
            prod[x] = input_df_agg_final.loc[input_df_agg_final['PART_12NC'] == x, "COMP_12NC"].to_list()
        return prod

    def Mtype_Dict_Preprocessing(self, input_df_agg):
        # Dict for Matertial Type
        material_12nc_type_list = [["PART_12NC", "TYPE"],
                                ["1_12NC_LIST", "1_CLASS"], 
                                ["2_12NC_LIST", "2_CLASS"], 
                                ["3_12NC_LIST", "3_CLASS"], 
                                ["4_12NC_LIST", "4_CLASS"]]
        def Merge(dict1, dict2):
            res = {**dict1, **dict2}
            return res

        Mtype = dict()
        for material_12nc_type in (material_12nc_type_list):
            sub_material = input_df_agg[material_12nc_type[0]].dropna()
            sub_type = input_df_agg[material_12nc_type[1]].dropna()
            sub_Mtype = dict(zip(sub_material,sub_type))
            Mtype = Merge(Mtype, sub_Mtype)
        
        return Mtype

        

### PUI Structure Rolling Process

In [ ]:
class PUI_Structure_Rolling:
    def __init__(self):
        self.PUI_anlyzied_slice_stack = pd.DataFrame([])
        self.df_all_valid_supply_path = pd.DataFrame([])
        self.PUI_stack = []

    def main(self, input_SC_dataframe, input_PUI_dataframe, input_PUI_ref_before_drop_multi_comp):

        print("Rolling Supply Path ===========================================================")
        for SC_Num, prod_item, consumed_item, consumed_plant, BOM_id in tqdm(zip(input_SC_dataframe["SC Index"], input_SC_dataframe["ITEM_NAME"], input_SC_dataframe["CONSUMED_ITEM"], input_SC_dataframe["PLANT"], input_SC_dataframe["ITEM_BOM_RT_ID"]), total = len(input_SC_dataframe)):
            all_paths_list = self.find_unique_path(prod, prod_item, consumed_item) # Get all applicable paths (prod to comp) -> list
            if len(all_paths_list) == 0:
                continue
            self.df_all_valid_supply_path = pd.concat([self.df_all_valid_supply_path, self.Mtype_Aggregation(SC_Num, all_paths_list)]) # Get all applicable supply chain with Mtype Mapping -> dataframe
            df_SC_analyezd_Stack = self.SC_Structure_Extraction(input_PUI_dataframe, all_paths_list, consumed_plant, BOM_id) # Extract each supply path and check if the supply path is valid
            
            #Result concating for each input supply path combination
            df_SC_analyezd_Stack["SC Index"] = SC_Num
            self.PUI_anlyzied_slice_stack = pd.concat([self.PUI_anlyzied_slice_stack, df_SC_analyezd_Stack]) 

        if len(self.PUI_anlyzied_slice_stack) != 0:
            self.PUI_anlyzied_slice_stack["Multi Chip"] = self.PUI_anlyzied_slice_stack["ITEM_BOM_RT_ID"].map(MultiChip_dict)
            self.PUI_anlyzied_slice_stack["Produced_Item_Category"] = self.PUI_anlyzied_slice_stack["Produced"].map(Mtype)
            self.PUI_anlyzied_slice_stack["Consumed_Item_Category"] = self.PUI_anlyzied_slice_stack["Consumed"].map(Mtype)
            self.PUI_anlyzied_slice_stack.insert(0, 'SC Index', self.PUI_anlyzied_slice_stack.pop('SC Index'))
            self.PUI_anlyzied_slice_stack.insert(2, 'ITEM_BOM_RT_ID', self.PUI_anlyzied_slice_stack.pop('ITEM_BOM_RT_ID'))
            self.PUI_anlyzied_slice_stack.insert(3, "Multi Chip", self.PUI_anlyzied_slice_stack.pop("Multi Chip"))
            self.PUI_anlyzied_slice_stack.insert(5, "Produced_Item_Category", self.PUI_anlyzied_slice_stack.pop("Produced_Item_Category"))
            self.PUI_anlyzied_slice_stack.insert(7, "Consumed_Item_Category", self.PUI_anlyzied_slice_stack.pop("Consumed_Item_Category"))

        print("\nMulti-Component Checking ======================================================")
        df_component_analyzed_result  = self.MultiComponent_Checking(self.PUI_anlyzied_slice_stack, input_SC_dataframe)

        print("\nPUI Mapping Result ============================================================")
        df_PUI_mapping = self.PUI_result_mapping(input_PUI_ref_before_drop_multi_comp, df_component_analyzed_result)
        
        #Result proprocessing
        if len(df_PUI_mapping) != 0:
            # Mapping JDA columns into PUI: ["JDA_PRODUCED_LOC_2", "JDA_CONSUMED_LOC_2", "FINAL START DATE", "FINAL END DATE"]
            input_SC_dataframe_to_be_mapped = input_SC_dataframe[["SC Index", "JDA_PRODUCED_LOC_2", "JDA_CONSUMED_LOC_2", "FINAL START DATE", "FINAL END DATE"]]
            df_PUI_mapping = df_PUI_mapping.merge(input_SC_dataframe_to_be_mapped, on='SC Index', how='left')

            # Column format and order change
            df_PUI_mapping["PART_12NC"] = df_PUI_mapping["PART_12NC"].astype(str)
            df_PUI_mapping["COMP_12NC"] = df_PUI_mapping["COMP_12NC"].astype(str)
            df_PUI_mapping.insert(0, 'PUI_Index', df_PUI_mapping.pop('PUI_Index'))
            df_PUI_mapping.insert(1, 'SC Index', df_PUI_mapping.pop('SC Index'))
            df_PUI_mapping.insert(2, 'ITEM_BOM_RT_ID', df_PUI_mapping.pop('ITEM_BOM_RT_ID'))
            df_PUI_mapping.insert(3, 'JDA_PRODUCED_LOC_2', df_PUI_mapping.pop('JDA_PRODUCED_LOC_2'))
            df_PUI_mapping.insert(4, 'JDA_CONSUMED_LOC_2', df_PUI_mapping.pop('JDA_CONSUMED_LOC_2'))
            df_PUI_mapping.insert(5, 'FINAL START DATE', df_PUI_mapping.pop('FINAL START DATE'))
            df_PUI_mapping.insert(6, 'FINAL END DATE', df_PUI_mapping.pop('FINAL END DATE'))
            df_PUI_mapping.insert(10, 'COMP_12NC', df_PUI_mapping.pop('COMP_12NC'))

            # Drop unneccsary columns
            df_PUI_mapping.drop(columns = ["SingleComp_index", "MutiComp", "MutiComp_index", "EFF_FR_DATE", "EFF_TO_DATE"], inplace = True)
            df_PUI_mapping.reset_index(drop= True, inplace = True)

        #Mapping two results back to JDA input
        input_SC_dataframe["Connected Rolling Structure"] = "No"
        input_SC_dataframe["Valid Rolling Structure"] = "No"
        if len(df_component_analyzed_result) != 0:
            input_SC_dataframe.loc[input_SC_dataframe["ITEM_BOM_RT_ID"].isin(df_component_analyzed_result["ITEM_BOM_RT_ID"]), "Connected Rolling Structure"] = "Yes"
            input_SC_dataframe.loc[input_SC_dataframe["ITEM_BOM_RT_ID"].isin(df_component_analyzed_result.loc[df_component_analyzed_result["Valid_Comp_Combination"] == True, "ITEM_BOM_RT_ID"].unique()), "Valid Rolling Structure"] = "Yes"

        return df_PUI_mapping, df_component_analyzed_result, self.df_all_valid_supply_path, input_SC_dataframe

    def find_all_paths(self, graph, start, end, path=None, unique_paths=None):
        if path is None:
            path = []
        if unique_paths is None:
            unique_paths = set()
        path.append(start)
        if start == end:
            # Convert path to a tuple so it can be added to a set
            unique_paths.add(tuple(path))
        else:
            for node in graph.get(start, []):
                if node not in path:  # Avoid cycles
                    self.find_all_paths(graph, node, end, path.copy(), unique_paths)
        return unique_paths


    def find_unique_path(self, graph, start, end, path=None, unique_paths=None):
        # Find all paths
        all_paths = self.find_all_paths(graph, start, end)
        # Convert each tuple path back to a list if needed
        all_paths = [list(path) for path in all_paths]
        return all_paths


    def Mtype_Aggregation(self, SC_Num, input_all_paths):
        Info = []
        for x in input_all_paths:
            Info_sub = []
            for y in x:
                Info_sub.append(Mtype[y])
            Info.append(Info_sub)

        #Output
        stacker = []
        for x, y in zip(Info, input_all_paths):
            stacker.append(x)
            stacker.append(y)
        output_df = pd.DataFrame(stacker)
        output_df.columns = [('Element_' + str(x + 1)) for x in range(len(output_df.columns))]
        output_df["SC Index"] = SC_Num
        output_df.insert(0, 'SC Index', output_df.pop('SC Index'))
        return output_df

    def SC_Structure_Extraction(self, input_PUI_ref, PUI_valid_path_list, consumed_plant, BOM_id):
        SC_Stack = []
        idx = 0
        
        for x in PUI_valid_path_list:
            idx +=1
            for y in range(len(x)):
                try:
                    SC_Stack.append([idx, x[y], x[y+1], input_PUI_ref.loc[(input_PUI_ref["PART_12NC"] == x[y]) & (input_PUI_ref["COMP_12NC"] == x[y+1]), "PLANT"].unique()])
                except:
                    # SC_Stack.append(["", "", "", ""])
                    continue

            
        df_SC_analyezd_Stack = pd.DataFrame(SC_Stack, columns = ["Rolling Result - Combination", "Produced", "Consumed", "Site"])
        #Check if target plant is in Available Plant for each combination
        df_SC_analyezd_Stack["Target Plant"] = consumed_plant
        df_SC_analyezd_Stack["ITEM_BOM_RT_ID"] = BOM_id
        df_SC_analyezd_Stack["Available Plant"] = df_SC_analyezd_Stack.apply(lambda x: True if consumed_plant in x["Site"] else False, axis = 1)


        for c in df_SC_analyezd_Stack["Rolling Result - Combination"].unique():
            if c != '':
                each_comb = df_SC_analyezd_Stack.loc[(df_SC_analyezd_Stack["Rolling Result - Combination"] == c)]
                if (each_comb["Available Plant"] == False).any() == True:
                    df_SC_analyezd_Stack.loc[(df_SC_analyezd_Stack["Rolling Result - Combination"] == c), "Valid_Plant"] = False
                else:
                    df_SC_analyezd_Stack.loc[(df_SC_analyezd_Stack["Rolling Result - Combination"] == c), "Valid_Plant"] = True

        df_SC_analyezd_Stack.fillna("", inplace = True)
        return df_SC_analyezd_Stack

    def MultiComponent_Checking(self, input_PUI_anlyzied_slice_stack, input_SC_dataframe):
        if len(input_PUI_anlyzied_slice_stack) == 0:
            return pd.DataFrame([])
        else:
            input_PUI_anlyzied_slice_stack["Valid_Comp_Combination"] = False
            # Initialize all records are invalid in the beginning 
            for bom_id in tqdm(input_PUI_anlyzied_slice_stack["ITEM_BOM_RT_ID"].unique(), total = len(input_PUI_anlyzied_slice_stack["ITEM_BOM_RT_ID"].unique())):
                df_extract_from_bom_id = input_PUI_anlyzied_slice_stack.loc[(input_PUI_anlyzied_slice_stack["ITEM_BOM_RT_ID"] == bom_id)]
                
                # Check if the provided number of SC != the number of rolling out SC Index
                if len(df_extract_from_bom_id["SC Index"].unique()) != len(input_SC_dataframe.loc[input_SC_dataframe["ITEM_BOM_RT_ID"] == bom_id]):
                    input_PUI_anlyzied_slice_stack.loc[(input_PUI_anlyzied_slice_stack["ITEM_BOM_RT_ID"] == bom_id), "Valid_Comp_Combination"] = False
                    continue
                else:
                    if sum(df_extract_from_bom_id["Multi Chip"] == "No") == 0:
                        #Check if it marked as Mutichip but only one SC is input
                        if len(input_SC_dataframe.loc[input_SC_dataframe["ITEM_BOM_RT_ID"] == bom_id]) == 1:
                            continue

                        # Get count of ICAM
                        ICAM_count_per_SC_index = df_extract_from_bom_id.loc[df_extract_from_bom_id["Produced_Item_Category"] =="ICAM", ["SC Index", "Produced"]].drop_duplicates().value_counts("Produced")
                        Valid_ICAM_list = ICAM_count_per_SC_index[ICAM_count_per_SC_index >= len(input_SC_dataframe.loc[input_SC_dataframe["ITEM_BOM_RT_ID"] == bom_id])].keys().to_list()
                        for valid_ICAM in Valid_ICAM_list:
                            # Get all valid ICAM combination (SC_Index, Combination)
                            valid_ICAM_comb = df_extract_from_bom_id.loc[df_extract_from_bom_id["Produced"] == valid_ICAM, ["SC Index", "Rolling Result - Combination"]]
                            valid_ICAM_comb_mapping_result = df_extract_from_bom_id.merge(valid_ICAM_comb, how='inner', on = ["SC Index", "Rolling Result - Combination"])
                            Checking_list_per_SC_index = []
                            
                            # Check if per SC combination has its valid ICAM
                            for val_plant_checking_per_SC_index in valid_ICAM_comb_mapping_result["SC Index"].unique():
                                if sum(valid_ICAM_comb_mapping_result.loc[(valid_ICAM_comb_mapping_result["SC Index"] == val_plant_checking_per_SC_index), "Valid_Plant"] == True) != 0:
                                    if val_plant_checking_per_SC_index not in Checking_list_per_SC_index:
                                        Checking_list_per_SC_index.append(val_plant_checking_per_SC_index)
                            
                            if len(Checking_list_per_SC_index) == len(valid_ICAM_comb["SC Index"].unique()):
                                for sc_idx, roll_comb in valid_ICAM_comb.values:
                                    if sum(valid_ICAM_comb_mapping_result.loc[(valid_ICAM_comb_mapping_result["SC Index"] == sc_idx) & (valid_ICAM_comb_mapping_result["Rolling Result - Combination"] == roll_comb), "Valid_Plant"] == False) == 0:
                                        input_PUI_anlyzied_slice_stack.loc[(input_PUI_anlyzied_slice_stack["SC Index"] == sc_idx) & (input_PUI_anlyzied_slice_stack["Rolling Result - Combination"] == roll_comb), "Valid_Comp_Combination"] = True
                    else:
                        for comb_id in df_extract_from_bom_id["Rolling Result - Combination"].unique():
                            df_extract_from_bom_id_and_comb_id = df_extract_from_bom_id[df_extract_from_bom_id["Rolling Result - Combination"] == comb_id]
                            if (df_extract_from_bom_id_and_comb_id["Valid_Plant"] == False).any():
                                input_PUI_anlyzied_slice_stack.loc[(input_PUI_anlyzied_slice_stack["ITEM_BOM_RT_ID"] == bom_id) & (input_PUI_anlyzied_slice_stack["Rolling Result - Combination"] == comb_id), "Valid_Comp_Combination"] = False
                            else:
                                input_PUI_anlyzied_slice_stack.loc[(input_PUI_anlyzied_slice_stack["ITEM_BOM_RT_ID"] == bom_id) & (input_PUI_anlyzied_slice_stack["Rolling Result - Combination"] == comb_id), "Valid_Comp_Combination"] = True
            return input_PUI_anlyzied_slice_stack

    def PUI_result_mapping(self, input_PUI_ref_before_drop_multi_comp, input_SC_analyezd_Stack):
        if len(input_SC_analyezd_Stack) == 0:
            return pd.DataFrame([])
        else:
            for idx, bom_id, p, c, plt, val in tqdm(zip(input_SC_analyezd_Stack["SC Index"], input_SC_analyezd_Stack["ITEM_BOM_RT_ID"], input_SC_analyezd_Stack["Produced"], input_SC_analyezd_Stack["Consumed"], input_SC_analyezd_Stack["Target Plant"], input_SC_analyezd_Stack["Valid_Comp_Combination"]), total = len(input_SC_analyezd_Stack["SC Index"])):
                if val == True:
                    sub_PUI_index_single = input_PUI_ref_before_drop_multi_comp.loc[(input_PUI_ref_before_drop_multi_comp["PART_12NC"] == p) & (input_PUI_ref_before_drop_multi_comp["COMP_12NC"] == str(c)) & (input_PUI_ref_before_drop_multi_comp["PLANT"] == plt), "SingleComp_index"].dropna().values
                    sub_PUI_index_multi = input_PUI_ref_before_drop_multi_comp.loc[(input_PUI_ref_before_drop_multi_comp["PART_12NC"] == p) & (input_PUI_ref_before_drop_multi_comp["COMP_12NC"] == str(c)) & (input_PUI_ref_before_drop_multi_comp["PLANT"] == plt), "MutiComp_index"].dropna().values
                    sub_PUI_index = set(sub_PUI_index_single).union(set(sub_PUI_index_multi))

                    sub_PUI = input_PUI_ref_before_drop_multi_comp.loc[sub_PUI_index]
                    sub_PUI["SC Index"] = idx
                    sub_PUI["ITEM_BOM_RT_ID"] = bom_id
                    
                   
                    if len(sub_PUI_index) != 0:
                        self.PUI_stack.append(sub_PUI)

            if len(self.PUI_stack) == 0:
                return pd.DataFrame([])
            else:
                PUI_mapping_result = pd.concat(self.PUI_stack)
                # Create an index for PUImapping result checking but keep the original PUI index
                PUI_mapping_result["PUI_Index"] = PUI_mapping_result.index
                PUI_mapping_result["Index for Mapping Result Validation"] = np.arange(1, len(PUI_mapping_result) + 1) 
                PUI_mapping_result.to_csv("PUI_Mapping_Checking_20240812.csv")
                print("\nPUI Mapping Result Validation =================================================")
                return self.PUI_result_mapping_validate(PUI_mapping_result)

    def PUI_result_mapping_validate(self, input_PUI_mapping_result):
        all_index_to_be_removed = []
        multi_comp_bom_id_list= input_PUI_mapping_result.loc[input_PUI_mapping_result["COMP_12NC"].str.contains(","), "ITEM_BOM_RT_ID"].unique()
        for x in tqdm(multi_comp_bom_id_list, total = len(multi_comp_bom_id_list)):
            removed_index_per_bom_id = []
            s = input_PUI_mapping_result.loc[(input_PUI_mapping_result["ITEM_BOM_RT_ID"] == x) & (input_PUI_mapping_result["TYPE"] == "ICAM") & (input_PUI_mapping_result["COMP_12NC"].str.contains(",")) & (input_PUI_mapping_result["1_CLASS"] == "CESF")]
            s_dict = s["COMP_12NC"].value_counts().to_dict()
            for (k, v) in s_dict.items():
                if (v % len(k.split(","))) != 0:
                    removed_index_per_bom_id.append(s.loc[s["COMP_12NC"] == k, "Index for Mapping Result Validation"].tolist())
            removed__all_index_per_bom_id = list(itertools.chain(*removed_index_per_bom_id))
            if len(removed__all_index_per_bom_id) != 0:
                all_index_to_be_removed.append(removed__all_index_per_bom_id)
        return input_PUI_mapping_result.loc[~input_PUI_mapping_result["Index for Mapping Result Validation"].isin(list(itertools.chain(*all_index_to_be_removed)))].drop("Index for Mapping Result Validation", axis = 1)


### Turnkey Check after Rolling and Broken Flow Checking for the input 12NCs

In [ ]:
class Turnkey_Broken_Validation:
    def __init__(self):
        self.turnkey_mtype_list = ["SLPD", "CEPT"]
        self.turnkey_list_all = []
        self.broken_list_dict = {}

    def main(self, input_JDA_frame_origin, input_JDA_frame_aggregated, input_rolling_map):
        print("\nTurnkey Checking ... ==========================================================")
        input_turnkey_frame = self.Turnkey_Checking(input_rolling_map)
        print("\nBroken Flow Checking ... ======================================================")
        input_broken_dict = self.Broken_Flow_Checking(input_JDA_frame_origin)

        aggregate_frame = input_JDA_frame_aggregated.merge(input_turnkey_frame, on='SC Index', how='left')
        input_JDA_frame_origin["Broken_Flow"] = input_JDA_frame_origin["SC Index"].map(input_broken_dict)
        input_JDA_frame_origin = input_JDA_frame_origin[["SC Index", "Broken_Flow", "ITEM_NAME", "ITEM_CATEGORY", "JDA_PRODUCED_LOC_2", "CONSUMED_ITEM", "CONSUMED_ITEM_CATEGORY", "JDA_CONSUMED_LOC_2", "RAW_PLANT"]]
        input_JDA_frame_origin.rename(columns = {"RAW_PLANT":"RAW_PLANT (CONSUMED)", }, inplace =True)
        return aggregate_frame, input_JDA_frame_origin

    def Turnkey_Checking(self, input_rolling_map):
        for SC_idx in tqdm(input_rolling_map.loc[input_rolling_map["Valid_Comp_Combination"] == True, "SC Index"].unique()):
            SC_valid_rolling_frame = input_rolling_map.loc[input_rolling_map["SC Index"] == SC_idx]
            SC_valid_rolling_frame_list = SC_valid_rolling_frame[["Produced", "Produced_Item_Category"]].values.tolist() + SC_valid_rolling_frame[["Consumed", "Consumed_Item_Category"]].values.tolist()           
            
            turnkey_indicator = False
            # Check if turnkey the predefined turnkey is included and not the produced/consumed items
            for turnkey_mtype in self.turnkey_mtype_list:
                if turnkey_mtype in list(itertools.chain(*SC_valid_rolling_frame_list)):
                    if turnkey_mtype not in [SC_valid_rolling_frame_list[0][1], SC_valid_rolling_frame_list[-1][1]]:
                        turnkey_indicator = True
                        break

            if turnkey_indicator == True:
                turnkey_frame = pd.DataFrame(SC_valid_rolling_frame_list, columns = ["12NC", "Mtype"])
                turnkey_SLPD_set = set(turnkey_frame.loc[turnkey_frame["Mtype"] == "SLPD", "12NC"])
                turnkey_CEPT_set = set(turnkey_frame.loc[turnkey_frame["Mtype"] == "CEPT", "12NC"])

                turnkey_SLPD_str = ", ".join(map(str, turnkey_SLPD_set))
                turnkey_CEPT_str = ", ".join(map(str, turnkey_CEPT_set))
                self.turnkey_list_all.append([SC_idx, turnkey_SLPD_str, turnkey_CEPT_str])
                
            else:
                continue
        return pd.DataFrame(self.turnkey_list_all, columns = ["SC Index", "Turnkey_SLPD", "Turnkey_CEPT"])
    
    def Broken_Flow_Checking(self, input_JDA_frame_origin):
        for SC_idx, prod_12NC, prod_Mtype, prod_loc in tqdm(zip(input_JDA_frame_origin["SC Index"], input_JDA_frame_origin["ITEM_NAME"], input_JDA_frame_origin["ITEM_CATEGORY"], input_JDA_frame_origin["JDA_PRODUCED_LOC_2"]), total = len(input_JDA_frame_origin)):
            if prod_Mtype in self.turnkey_mtype_list:
                self.broken_list_dict[SC_idx] = False
            else:
                if len(input_JDA_frame_origin.loc[(input_JDA_frame_origin["CONSUMED_ITEM"] == prod_12NC) & (input_JDA_frame_origin["JDA_CONSUMED_LOC_2"] == prod_loc)]) != 0:
                    self.broken_list_dict[SC_idx] = True
                else:
                    self.broken_list_dict[SC_idx] = False
        return self.broken_list_dict

### Result Checking and Exporting

In [ ]:
from styleframe import StyleFrame, Styler, utils

class General_function:
     def __init__(self):
          self.today = (datetime.datetime.today()).strftime('%Y%m%d')
          self.output_path = "./Output/Result_table_{}.xlsx".format(self.today)

     def Export_to_excel_with_style(self, input_Result, input_Result_SP, input_Reuslt_PUI_anlyzied_slice, input_Result_Broken_Flow_source, input_Result_JDA_source):
          try:
               input_Result_drop_dup = input_Result.drop(columns=["SC Index", "ITEM_BOM_RT_ID"]).drop_duplicates().sort_values(by = "PUI_Index")
          except:
               input_Result_drop_dup = input_Result.copy()

          Result_drop_dup_frame = General_function().style_changes(input_Result_drop_dup, 
                                                          [["PUI_Index"], ["COMP_12NC"], ["PLANT"], ["JDA_PRODUCED_LOC_2", "JDA_CONSUMED_LOC_2", "FINAL START DATE", "FINAL END DATE"]],
                                                          ["yellow", "#85C1E9", "green", "#ADADAD"]
                                                          )
          
          Result_frame = General_function().style_changes(input_Result, 
                                                          [["PUI_Index", "SC Index", "ITEM_BOM_RT_ID"], ["COMP_12NC"], ["PLANT"], ["JDA_PRODUCED_LOC_2", "JDA_CONSUMED_LOC_2", "FINAL START DATE", "FINAL END DATE"]],
                                                          ["yellow", "#85C1E9", "green", "#ADADAD"]
                                                          )
          Result_SP_frame = General_function().style_changes(input_Result_SP.reset_index(drop = True),
                                                            [
                                                               ["SC Index", "Rolling Result - Combination"], 
                                                               ["Produced", "Produced_Item_Category", "Consumed", "Consumed_Item_Category"], 
                                                               ["Target Plant"], 
                                                               ["ITEM_BOM_RT_ID"]
                                                            ],
                                                            ["yellow", "#85C1E9", "green", ]
                                                          )

          Reuslt_PUI_anlyzied_slice_frame = General_function().style_changes(input_Reuslt_PUI_anlyzied_slice.reset_index(drop = True),
                                                          [["SC Index"]],
                                                          ["yellow"]
                                                          )
          Result_Broken_Flow_source_frame = General_function().style_changes(input_Result_Broken_Flow_source,
                                                          [["SC Index", "Broken_Flow", "RAW_PLANT (CONSUMED)"]],
                                                          ["yellow"]
                                                          )
          input_Result_JDA_source_frame = General_function().style_changes(input_Result_JDA_source,
                                                            [
                                                                 ["SC Index", "Connected Rolling Structure", "Valid Rolling Structure"], 
                                                                 ["ITEM_NAME", "ITEM_CATEGORY", "ITEM_DESCRIPTION", "PRODUCED_LOCATION", "JDA_PRODUCED_LOC_2"],
                                                                 ["CONSUMED_ITEM", "CONSUMED_ITEM_CATEGORY", "CONSUMED_ITEM_DESCRIPTION", "RAW_PLANT", "PLANT", "JDA_CONSUMED_LOC_2"],
                                                                 ["Excluded"],
                                                                 ["Turnkey_SLPD", "Turnkey_CEPT"],
                                                                 ["FINAL START DATE","FINAL END DATE"]
                                                             ],
                                                             ["yellow", "#85C1E9", "green", "#7D7DFF", "#1AFD9C", "#ADADAD"]
                                                            )
          

          #Export to excel
          excel_writer = StyleFrame.ExcelWriter(self.output_path)
          Result_drop_dup_frame.to_excel(excel_writer, sheet_name = "Result - PUI Mapping (Drop_dup)", index= False, header = True)
          Result_frame.to_excel(excel_writer, sheet_name = "Result - PUI Mapping", index= False, header = True)
          Result_SP_frame.to_excel(excel_writer, sheet_name = "Analyzed - Rolling Supply Path", index= False, header = True)
          Reuslt_PUI_anlyzied_slice_frame.to_excel(excel_writer, sheet_name = "Initial - Rolling Supply Chain", index= False, header = True)
          Result_Broken_Flow_source_frame.to_excel(excel_writer, sheet_name = "Initial - Broken Flow Checking", index= False, header = True)
          input_Result_JDA_source_frame.to_excel(excel_writer, sheet_name = "Input - JDA Source", index= False, header = True)
          excel_writer.save()

          return "Data exported completedly !"
          
     def style_changes(self, input_frame, update_columns_combination, color_combination):
          # Reformat and export the excel file with color and font style
          if len(input_frame) == 0:
               return StyleFrame(pd.DataFrame([]))
          else:
               sf_output_ref = StyleFrame(input_frame)
               for h_col in input_frame.columns:
                    sf_output_ref[h_col] = input_frame[h_col].fillna("").values
                    sf_output_ref[h_col] = sf_output_ref[h_col].astype(str)
                    header_width = input_frame[h_col].astype(str).str.len().max() + 8
                    if header_width <= len(h_col):
                         header_width = len(h_col) + 12
                    sf_output_ref.apply_column_style(cols_to_style = h_col,
                                                  styler_obj=Styler(font=utils.fonts.calibri, font_size= 11),
                                                  width=header_width,
                                                  style_header=False
                                                  )

                    
               sf_output_ref.apply_headers_style(styler_obj = Styler(font = 'Calibri', font_size = 11, bg_color = "#FDEBD0"), cols_to_style = input_frame.columns)
               for col_list, color in zip(update_columns_combination, color_combination):
                    sf_output_ref.apply_headers_style(styler_obj = Styler(font = 'Calibri', font_size = 11, bg_color = color), cols_to_style = col_list)
               return sf_output_ref     


### Execution  - Trigger all Flows Working 

In [ ]:

if __name__=='__main__':
    prod, MultiChip_dict, df_JDA_origin, df_JDA_input, df_PUI_raw, df_agg, df_agg_final, Mtype = Data_preprocessing().main()
    Result, Result_SP, Reuslt_PUI_anlyzied_slice, Result_JDA_source = PUI_Structure_Rolling().main(df_JDA_input, df_agg_final, df_agg)
    Result_JDA_output, Result_JDA_frame_broken= Turnkey_Broken_Validation().main(df_JDA_origin, df_JDA_input, Result_SP)

    # Export to excel with style
    print("\n>> Result is now exporting to excel files ...... ")
    Excel_Exportint_Start_Time = time.time()
    General_function().Export_to_excel_with_style(Result, Result_SP, Reuslt_PUI_anlyzied_slice, Result_JDA_frame_broken, Result_JDA_output)
    print(">> Export Process is now completed in {} mins!".format(round((time.time()-Excel_Exportint_Start_Time)/60, 2)))

    os.startfile(os.getcwd().replace("\\", "/") + "/Output/")



### Completion Message

In [ ]:
print("Analysis Process End ! Time spent: {time_spent} mins".format(time_spent = round((time.time()-Program_Start_Time)/60, 2)))

